In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [3]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"]
    .transform(lambda x: x / x.max())
).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [4]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [5]:
ema_ord["acumulado_combinado"].describe()

count    431872.000000
mean          0.978025
std           0.089885
min           0.010000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: acumulado_combinado, dtype: float64

In [6]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL NORTE") & (ema_ord["product_ref"]=="DAE02286025")]

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
284265,SUCURSAL NORTE,DAE02286025,2025.0,38,1.639138,97279.53,159454.57,0.0007,0.2257,1.388627e+10,87086.03,0.52,0.0,5505434.99,0.0,0.37,AAA
284266,SUCURSAL NORTE,DAE02286025,2025.0,39,1.311310,97279.53,127563.62,0.0006,0.2710,1.122560e+05,0.88,0.57,12.0,6397786.87,1.0,0.63,A
284267,SUCURSAL NORTE,DAE02286025,2025.0,40,1.049048,97279.53,102050.90,0.0004,0.3224,1.122560e+05,1.10,0.63,13.0,8917940.07,1.0,0.68,A
284268,SUCURSAL NORTE,DAE02286025,2025.0,41,0.839238,97279.53,81640.68,0.0004,0.3860,9.960160e+04,1.22,0.68,14.0,11396412.15,1.0,0.73,A
284269,SUCURSAL NORTE,DAE02286025,2025.0,42,0.671390,97279.53,65312.50,0.0003,0.4443,8.425310e+04,1.29,0.73,15.0,13257994.93,1.0,0.78,A
284254,SUCURSAL NORTE,DAE02286025,2025.0,27,0.320000,97279.53,31129.45,0.0001,0.6532,0.000000e+00,0.00,0.90,0.0,21817210.61,0.0,0.91,B
284255,SUCURSAL NORTE,DAE02286025,2025.0,28,0.256000,97279.53,24903.56,0.0001,0.7052,0.000000e+00,0.00,0.90,0.0,24818051.59,0.0,0.93,B
284256,SUCURSAL NORTE,DAE02286025,2025.0,29,0.204800,97279.53,19922.85,0.0001,0.7601,9.016052e+08,45254.83,0.90,0.0,30779504.19,0.0,0.96,C
284257,SUCURSAL NORTE,DAE02286025,2025.0,30,0.184320,97279.53,17930.56,0.0001,0.7857,1.034923e+09,57718.40,0.90,0.0,32963950.08,0.0,0.97,C
284258,SUCURSAL NORTE,DAE02286025,2025.0,31,0.165888,97279.53,16137.51,0.0001,0.8142,9.766749e+08,60522.03,0.90,0.0,34221532.48,0.0,0.97,C


## NIVEL DE SERVICIO

In [7]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.98, 0.95, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [8]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
205638,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,66.404983,13771.02,914464.35,0.0009,0.0989,201182.2,0.22,0.00,1.976048,0.22,0.8,0.22,AAA,0.999
205639,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,59.323986,13771.02,816951.80,0.0008,0.1056,155220.8,0.19,0.00,3.092958,0.41,1.0,0.26,AAA,0.999
205640,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,34,56.259189,13771.02,774746.42,0.0008,0.1144,154949.3,0.20,0.01,3.318653,0.61,1.0,0.28,AAA,0.999
205647,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,55.645908,13771.02,766300.91,0.0008,0.1176,145597.2,0.19,0.01,3.990654,0.80,1.0,0.28,AAA,0.999
205633,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,27,54.093333,13771.02,744920.37,0.0008,0.1248,0.0,0.00,0.01,0.000000,0.80,0.0,0.05,AAA,0.999


In [9]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [10]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,1453,378,1656,50497
SUCURSAL BARRANQUILLA,1915,684,2180,49205
SUCURSAL BUCARAMANGA,2003,687,2198,49096
SUCURSAL CALI,1810,555,2224,49395
SUCURSAL CALLE 6,1789,757,2229,49209
SUCURSAL MEDELLIN,1554,528,1960,49942
SUCURSAL NORTE,1606,530,2008,49840
SUCURSAL VALLADOLID,1454,441,1909,50180
